# Optimize Hyper-Parameters

In [1]:
!pip install -q sagemaker-experiments==0.1.13

In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Specify the S3 Location of the Features

In [3]:
%store -r processed_train_data_s3_uri

In [4]:
print(processed_train_data_s3_uri)

s3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-train


In [5]:
%store -r processed_validation_data_s3_uri

In [6]:
print(processed_validation_data_s3_uri)

s3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-validation


In [7]:
%store -r processed_test_data_s3_uri

In [8]:
print(processed_test_data_s3_uri)

s3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-test


In [9]:
print(processed_train_data_s3_uri)
!aws s3 ls $processed_train_data_s3_uri/

s3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-train
2020-05-30 20:25:56      51032 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-05-30 20:25:51      72298 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [10]:
print(processed_validation_data_s3_uri)
!aws s3 ls $processed_validation_data_s3_uri/

s3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-validation
2020-05-30 20:25:57       3519 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-05-30 20:25:52       4243 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [11]:
print(processed_test_data_s3_uri)
!aws s3 ls $processed_test_data_s3_uri/

s3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-test
2020-05-30 20:25:57       3266 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-05-30 20:25:52       4476 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [12]:
s3_input_train_data = sagemaker.s3_input(s3_data=processed_train_data_s3_uri, 
                                         distribution='ShardedByS3Key') 
s3_input_validation_data = sagemaker.s3_input(s3_data=processed_validation_data_s3_uri, 
                                              distribution='ShardedByS3Key')
s3_input_test_data = sagemaker.s3_input(s3_data=processed_test_data_s3_uri, 
                                        distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-949806492859/sagemaker-scikit-learn-2020-05-30-20-21-46-715/output/bert-test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


In [13]:
!cat src/tf_bert_reviews.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.1.0.1.0.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'smdebug==0.7.2'])
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers.configuration_distilbert import DistilBertConfig


CLASSES = [1, 2, 3, 4, 5]


def select_data_and_label_from_record(record):
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids']
    }

    y = record['lab

# Setup Static Hyper-Parameters for Classification Layer
First, retrieve `max_seq_length` from the prepare phase.

In [14]:
%store -r max_seq_length

In [15]:
print(max_seq_length)

128


In [29]:
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=50
validation_steps=50
test_steps=50
train_instance_count=1
train_instance_type='ml.c5.4xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
freeze_bert_layer=False
input_mode='Pipe'
run_validation=True
run_test=True
run_sample_predictions=True

# Track the Optimizations Within our Experiment

In [17]:
%store -r experiment_name

In [18]:
print(experiment_name)

Amazon-Customer-Reviews-BERT-Experiment-1590873667


In [19]:
%store -r trial_name

In [20]:
print(trial_name)

trial-1590873667


In [21]:
import time
from smexperiments.trial import Trial

timestamp = '{}'.format(int(time.time()))

trial = Trial.load(trial_name=trial_name)
print(trial)

Trial(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f149a86fdd8>,trial_name='trial-1590873667',trial_arn='arn:aws:sagemaker:us-east-1:949806492859:experiment-trial/trial-1590873667',display_name='trial-1590873667',experiment_name='Amazon-Customer-Reviews-BERT-Experiment-1590873667',creation_time=datetime.datetime(2020, 5, 30, 21, 21, 7, 844000, tzinfo=tzlocal()),created_by={},last_modified_time=datetime.datetime(2020, 5, 30, 21, 41, 25, 618000, tzinfo=tzlocal()),last_modified_by={},response_metadata={'RequestId': 'ba1e5601-8094-4580-85aa-e25ccfea75d8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ba1e5601-8094-4580-85aa-e25ccfea75d8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '326', 'date': 'Sat, 30 May 2020 22:43:18 GMT'}, 'RetryAttempts': 0})


In [22]:
from smexperiments.tracker import Tracker

tracker_optimize = Tracker.create(display_name='optimize-1', 
                                  sagemaker_boto_client=sm)

optimize_trial_component_name = tracker_optimize.trial_component.trial_component_name
print('Optimize trial component name {}'.format(optimize_trial_component_name))

Optimize trial component name TrialComponent-2020-05-30-224319-ngbq


# Attach the `deploy` Trial Component and Tracker as a Component to the Trial

In [23]:
trial.add_trial_component(tracker_optimize.trial_component)

# Setup Dynamic Hyper-Parameter Ranges to Explore


In [24]:
from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import CategoricalParameter
from sagemaker.tuner import HyperparameterTuner
                                                
hyperparameter_ranges = {
    'epochs': IntegerParameter(1, 4, scaling_type='Logarithmic'),
    'learning_rate': ContinuousParameter(0.00001, 0.00005, scaling_type='Linear'),
}

# Track the Hyper-Parameter Ranges

In [25]:
tracker_optimize.log_parameters(hyperparameter_ranges)

# Setup Metrics

In [26]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

In [30]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                       source_dir='src',
                       role=role,
                       train_instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       train_instance_type=train_instance_type,
                       train_volume_size=train_volume_size,
                       py_version='py3',
                       framework_version='2.1.0',
                       hyperparameters={'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'max_seq_length': max_seq_length,
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
                       train_max_run=7200 # max 2 hours * 60 minutes seconds per hour * 60 seconds per minute
                      )

# Setup HyperparameterTuner with Estimator and Hyper-Parameter Ranges

In [31]:
objective_metric_name = 'validation:accuracy'

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_type='Maximize',
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metrics_definitions,
    max_jobs=2,
    max_parallel_jobs=1,
    strategy='Bayesian',
    early_stopping_type='Auto'
)

# Start Tuning Job

In [32]:
tuner.fit(inputs={'train': s3_input_train_data, 
                  'validation': s3_input_validation_data,
                  'test': s3_input_test_data
          }, 
          include_cls_metadata=False)

INFO:root:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: tensorflow-training-200530-2243


# Check Tuning Job Status
Re-run this cell to track the status.

In [33]:
from pprint import pprint

tuning_job_name = tuner.latest_tuning_job.job_name

job_description = sm.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = job_description['HyperParameterTuningJobStatus']

print('\n')
print(status)
print('\n')
pprint(job_description)

if status != 'Completed':
    job_count = job_description['TrainingJobStatusCounters']['Completed']
    print('Not yet complete, but {} jobs have completed.'.format(job_count))
    
    if job_description.get('BestTrainingJob', None):
        print("Best candidate:")
        pprint(job_description['BestTrainingJob']['TrainingJobName'])
        pprint(job_description['BestTrainingJob']['FinalHyperParameterTuningJobObjectiveMetric'])
    else:
        print("No training jobs have reported results yet.")    



InProgress


{'CreationTime': datetime.datetime(2020, 5, 30, 22, 43, 44, tzinfo=tzlocal()),
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:949806492859:hyper-parameter-tuning-job/tensorflow-training-200530-2243',
 'HyperParameterTuningJobConfig': {'HyperParameterTuningJobObjective': {'MetricName': 'validation:accuracy',
                                                                        'Type': 'Maximize'},
                                   'ParameterRanges': {'CategoricalParameterRanges': [],
                                                       'ContinuousParameterRanges': [{'MaxValue': '5e-05',
                                                                                      'MinValue': '1e-05',
                                                                                      'Name': 'learning_rate',
                                                                                      'ScalingType': 'Linear'}],
                                           

In [34]:
from IPython.core.display import display, HTML
    
display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/{}">Hyper-Parameter Tuning Job</a></b>'.format(region, tuning_job_name)))

In [35]:
from sagemaker.analytics import ExperimentAnalytics

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    metric_names=['validation:accuracy'],
    parameter_names=['max_seq_length'],
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_df = lineage_table.dataframe()
lineage_df.shape

(3, 10)

In [36]:
lineage_df

,TrialComponentName,DisplayName,SourceArn,max_seq_length,validation:accuracy - Min,validation:accuracy - Max,validation:accuracy - Avg,validation:accuracy - StdDev,validation:accuracy - Last,validation:accuracy - Count
0,TrialComponent-2020-05-30-212107-aneg,prepare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tensorflow-training-2020-05-30-21-31-31-022-aw...,train,arn:aws:sagemaker:us-east-1:949806492859:train...,128.0,0.5,0.5,0.5,0.0,0.5,1.0
2,TrialComponent-2020-05-30-224319-ngbq,optimize-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# _Please Wait for the ^^ Tuning Job ^^ to Complete Above_

In [37]:
tuner.wait()

...............................................................................................................................................................................................................................................................................!


# Show the Tuning Job
### _Note:  This will fail at first.  Please wait about 15-30 seconds and re-run._

In [38]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

hp_results = HyperparameterTuningJobAnalytics(
    sagemaker_session=sess, 
    hyperparameter_tuning_job_name=tuning_job_name
)

df_results = hp_results.dataframe()
df_results.shape

(2, 8)

In [39]:
df_results.sort_values('FinalObjectiveValue', ascending=0)

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,epochs,learning_rate
1,0.7143,524.0,2020-05-30 22:54:33+00:00,tensorflow-training-200530-2243-001-215ae3f3,Completed,2020-05-30 22:45:49+00:00,1.0,0.000030
0,0.5000,475.0,2020-05-30 23:06:33+00:00,tensorflow-training-200530-2243-002-91cba3c3,Stopped,2020-05-30 22:58:38+00:00,1.0,0.000011


# Show the Best Candidate

In [40]:
df_results.sort_values('FinalObjectiveValue', ascending=0).head(1)

,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,epochs,learning_rate
1,0.7143,524.0,2020-05-30 22:54:33+00:00,tensorflow-training-200530-2243-001-215ae3f3,Completed,2020-05-30 22:45:49+00:00,1.0,0.00003


# Plot Results Over Time
Let's plot how the objective metric changes over time.  This plots helps determine if the tuning jobs are sufficient for your parameter space.  

Note:  When using the Bayesian strategy, you may see unsteady progress as the algorithm balances exploration of new parameter combinations vs. exploitation of known-good parameter combinations.  In general, you will see a general trend towards better results.

In [43]:
# import bokeh
# import bokeh.io
# bokeh.io.output_notebook()
# from bokeh.plotting import figure, show
# from bokeh.models import HoverTool

# class HoverHelper():

#     def __init__(self, tuning_analytics):
#         self.tuner = tuning_analytics

#     def hovertool(self):
#         tooltips = [
#             ("FinalObjectiveValue", "@FinalObjectiveValue"),
#             ("TrainingJobName", "@TrainingJobName"),
#         ]
#         for k in self.tuner.hyperparameter_ranges().keys():
#             tooltips.append( (k, "@{%s}" % k) )

#         ht = HoverTool(tooltips=tooltips)
#         return ht

#     def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
#         return [self.hovertool(), standard_tools]

# hover = HoverHelper(tuner)

# p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
# p.circle(source=df_results, x='TrainingStartTime', y='FinalObjectiveValue')
# show(p)

Loading BokehJS ...

# Analyze the Correlation Between Each Parameter and Your Objective Metric
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [49]:
# ranges = tuner.hyperparameter_ranges()
# figures = []
# for hp_name, hp_range in ranges.items():
#     print(hp_range)
#     categorical_args = {}
#     if hp_range.get('Values'):
#         # This is marked as categorical.  Check if all options are actually numbers.
#         def is_num(x):
#             try:
#                 float(x)
#                 return 1
#             except:
#                 return 0           
#         vals = hp_range['Values']
#         if sum([is_num(x) for x in vals]) == len(vals):
#             # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
#             print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
#         else:
#             # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
#             categorical_args['x_range'] = vals

#     # Now plot it
#     p = figure(plot_width=500, plot_height=500, 
#                title="Objective vs %s" % hp_name,
#                tools=hover.tools(),
#                x_axis_label=hp_name, y_axis_label=objective_name,
#                **categorical_args)
#     p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
#     figures.append(p)
# show(bokeh.layouts.Column(*figures))

[{'Name': 'learning_rate', 'MinValue': '1e-05', 'MaxValue': '5e-05', 'ScalingType': 'Linear'}]


TypeError: 'list' object is not callable

# Pass `tuning_job_name` to the Next Notebook

In [ ]:
%store tuning_job_name

In [ ]:
%store